# Rainfall Prediction in Melbourne Using Machine Learning

Predict whether it will rain today in the Melbourne region using historical weather data.

Dataset sourced from the Australian Bureau of Meteorology via Kaggle.
Filtered to Melbourne, Melbourne Airport, and Watsonia locations.

4. Methodology (VERY IMPORTANT)

Explain why, not just what:

Dropped high-missing-value features

Addressed data leakage by redefining target

Engineered seasonal feature

Used location-based filtering

Built ML pipeline with preprocessing

Applied GridSearchCV with stratified folds
